In [11]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

In [12]:
reelgo_2 = pd.read_csv('REELGO_2.CSV')
reelgo_3 = pd.read_csv('REELGO_3.CSV')

/var/folders/bm/7b2xvf295jl53415tpxdssxr0000gn/T/ipykernel_85627/1204800399.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  reelgo_3 = pd.read_csv('REELGO_3.CSV')


In [13]:
reelgo_2 = reelgo_2[reelgo_2['Type'] == 'tv']
reelgo_3 = reelgo_3[reelgo_3['Type'] == 'tv']

In [14]:
dummies = pd.get_dummies(reelgo_2['Service'], prefix = 'Service')
reelgo_2 = pd.concat([reelgo_2, dummies], axis=1)
reelgo_2 = reelgo_2[['Title', 'Genre', 'Service_netflix', 'Service_hulu_plus', 'Service_amazon_prime', 
                     'Service_hbo', 'Service_hbo_max']]

reelgo_2['IsPremiumPlatform'] = reelgo_2['Service_netflix'] + reelgo_2['Service_hulu_plus'] + \
reelgo_2['Service_amazon_prime'] + reelgo_2['Service_hbo'] + reelgo_2['Service_hbo_max']

reelgo_2['IsPremiumPlatform'] = reelgo_2['IsPremiumPlatform'].apply(lambda x: 1 if x > 0 else 0)

reelgo_2 = reelgo_2[['Title', 'Genre', 'IsPremiumPlatform']]
reelgo_2

,Title,Genre,IsPremiumPlatform
0,Breaking Bad,Crime,1
1,Breaking Bad,Drama,1
2,Game of Thrones,Drama,1
3,Game of Thrones,Action & Adventure,1
4,Game of Thrones,Fantasy,1
...,...,...,...
50671,Wendy,Children,1
50672,Wendy,Drama,0
50673,Wendy,Animation,0
50674,Wendy,Children,0


In [15]:
dummies = pd.get_dummies(reelgo_2['Genre'], prefix = 'Genre')
reelgo_2 = pd.concat([reelgo_2, dummies], axis=1)
reelgo_2 = reelgo_2.drop(columns=['Genre_Stand-up & Talk', 'Genre'])
reelgo_2 = reelgo_2.groupby('Title').sum().reset_index()
reelgo_2

,Title,IsPremiumPlatform,Genre_Action & Adventure,Genre_Animation,Genre_Anime,Genre_Biography,Genre_Children,Genre_Comedy,Genre_Crime,Genre_Cult,...,Genre_Horror,Genre_LGBTQ,Genre_Musical,Genre_Mystery,Genre_Reality,Genre_Romance,Genre_Science-Fiction,Genre_Sport,Genre_Thriller,Genre_Travel
0,#BlackLove,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,4,0,0,0,0
1,"#MeToo, Now What?",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,#RichKids of Beverly Hills,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,#blackAF,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,#killerpost,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10766,那年花開月正圓,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10767,阳关道,3,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
10768,頭文字D First Stage,3,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
10769,부릉! 부릉! 브루미즈,8,4,4,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
merge_df = pd.merge(
    reelgo_3,
    reelgo_2,
    on = 'Title',
    how = 'inner'
)

merge_df['AgeOfShow'] = 2020 - merge_df['Released Year']
dummies = pd.get_dummies(merge_df['Tag'], prefix = 'Tag')
merge_df = pd.concat([merge_df, dummies], axis=1)

dummies = pd.get_dummies(merge_df['AgeRating'], prefix = 'AgeRating')
merge_df = pd.concat([merge_df, dummies], axis=1)


merge_df = merge_df.drop(columns=['Unnamed: 0', 'Title', 'Type', 'Service', 'Tag', 'Tag_Drug', 'AgeRating_7+',
                                  'Seasons',
                                  'AgeRating',
                                  'Rated', 
                                  'Released Year', 
                                  'Duration Year'])

merge_df


#Dropped Columns from dummies - Travel (Genre), Drug (Tag), 'AgeRating_7+'

,IMDB,ReelGood,Rent or Buy Available,Exclusive Service,Has Tag,IsPremiumPlatform,Genre_Action & Adventure,Genre_Animation,Genre_Anime,Genre_Biography,...,Tag_Vampire,Tag_Video Games,Tag_War,Tag_World War II,Tag_Wrestling,Tag_Zombie,AgeRating_13+,AgeRating_16+,AgeRating_18+,AgeRating_all
0,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32590,NaN,NaN,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32591,NaN,16.0,1,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
32592,NaN,16.0,1,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
32593,NaN,16.0,1,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
merge_df = merge_df[merge_df['IMDB'].notnull()]
merge_df

,IMDB,ReelGood,Rent or Buy Available,Exclusive Service,Has Tag,IsPremiumPlatform,Genre_Action & Adventure,Genre_Animation,Genre_Anime,Genre_Biography,...,Tag_Vampire,Tag_Video Games,Tag_War,Tag_World War II,Tag_Wrestling,Tag_Zombie,AgeRating_13+,AgeRating_16+,AgeRating_18+,AgeRating_all
0,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,9.5,100.0,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32173,4.0,23.0,0,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32174,5.4,19.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32181,4.8,19.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32182,1.3,25.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
merge_df.isnull().sum()

IMDB                     0
ReelGood                 0
Rent or Buy Available    0
Exclusive Service        0
Has Tag                  0
                        ..
Tag_Zombie               0
AgeRating_13+            0
AgeRating_16+            0
AgeRating_18+            0
AgeRating_all            0
Length: 128, dtype: int64

In [19]:
y = merge_df.iloc[:, 0]
X = merge_df.iloc[:, 1:]

In [20]:
# Add interaction terms up to 2nd degree
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
feature_names = poly.get_feature_names(X.columns) # Get feature names

# Add constant term for intercept
X_poly = sm.add_constant(X_poly)

# Fit logistic regression model
model = sm.OLS(y, X_poly).fit()

# Print model summary with feature names
df = model.summary(xname=['const'] + feature_names)
print(model.summary(xname=['const'] + feature_names))

/Users/param/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


                            OLS Regression Results                            
Dep. Variable:                   IMDB   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     14.73
Date:                Mon, 24 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:46:51   Log-Likelihood:                -34478.
No. Observations:               31507   AIC:                         7.408e+04
Df Residuals:                   28943   BIC:                         9.551e+04
Df Model:                        2563                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [34]:
summary_df = pd.read_html(df.tables[1].as_html(), header=0, index_col=0)[0]
summary_df[((summary_df['P>|t|'] < 0.05) & (summary_df['coef'] < -0.1))]

,coef,std err,t,P>|t|,[0.025,0.975]
Rent or Buy Available,-0.1139,0.029,-3.920,0.000,-0.171,-0.057
Exclusive Service,-0.2181,0.034,-6.340,0.000,-0.286,-0.151
Genre_Action & Adventure,-0.1527,0.022,-6.895,0.000,-0.196,-0.109
Genre_Comedy,-0.2514,0.019,-13.529,0.000,-0.288,-0.215
Genre_Family,-0.1766,0.029,-6.054,0.000,-0.234,-0.119
Genre_Game Show,-0.4366,0.058,-7.492,0.000,-0.551,-0.322
Genre_Horror,-0.2112,0.075,-2.818,0.005,-0.358,-0.064
Genre_Reality,-0.1715,0.023,-7.397,0.000,-0.217,-0.126
Genre_Romance,-0.1724,0.059,-2.902,0.004,-0.289,-0.056
Genre_Science-Fiction,-0.3382,0.039,-8.738,0.000,-0.414,-0.262


In [38]:
pd.set_option('display.max_rows', 1000)
summary_df[((summary_df['P>|t|'] < 0.05) & (summary_df['coef'] > 0.1))]

,coef,std err,t,P>|t|,[0.025,0.975]
const,5.6865,0.084,67.783,0.000,5.522,5.851
Genre_Biography,0.3618,0.143,2.530,0.011,0.082,0.642
Genre_Documentary,0.2151,0.023,9.415,0.000,0.170,0.260
Genre_Food,0.2857,0.058,4.897,0.000,0.171,0.400
Genre_Home & Garden,0.2460,0.049,4.976,0.000,0.149,0.343
AgeRating_all,0.1071,0.048,2.219,0.026,0.013,0.202
Rent or Buy Available Genre_History,0.2671,0.039,6.907,0.000,0.191,0.343
Rent or Buy Available Genre_Romance,0.1599,0.025,6.485,0.000,0.112,0.208
Exclusive Service Genre_Documentary,0.2192,0.031,7.007,0.000,0.158,0.281
Exclusive Service Genre_Family,0.1047,0.040,2.610,0.009,0.026,0.183
